In [4]:
# making changes to tempdata

In [5]:
# make a working pipeline that you can apply and get a viz for every ten seconds.

# refactor readfilerun to take in code properly


In [6]:
import numpy as np
import json
import sys
import pandas as pd
import os
import glob
import re
import matplotlib.pyplot as plt
import seaborn as sns
from os import listdir
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingRegressor
import warnings

from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer

warnings.filterwarnings("ignore")

In [7]:
sys.path.insert(0, 'src')
from helper import *
from eda import *
from train import *
from etl import *

%load_ext autoreload

%autoreload 2

### etl (new readfilerun and gen)

In [8]:
'data/raw/train_r/20220116T055105_20-100-true-20-100-iperf.csv'.split('_')

['data/raw/train', 'r/20220116T055105', '20-100-true-20-100-iperf.csv']

In [9]:
# def readfilerun(run_):#, output_dir):
#     outdir = os.path.join(os.getcwd() , "data3", "tempdata3")
#     names = listdir('data3') # all filenames in data
#     daneruns = [x for x in names if not 'losslog' in x]
#     daneruns.remove('tempdata3')
    
#     # daneruns = ['data/20220116T055105_20-100-true-20-100-iperf.csv', 'data/20220116T055942_20-250-true-20-250-iperf.csv']
#     losslogs = [x for x in names if 'losslog' in x]
#     #print(losslogs)
#     for count, run in enumerate(daneruns):
#         print(count)
#         # print(run) #for debug
#         curr_losslog = losslogs[count]
#         print("run: ", run)
#         run_labels = run.split('_')[1].split('.')[0].split('-')
#         print("run_labels:", run_labels)
#         temp_label_str = '-'.join(run_labels) 
#         #print(temp_label_str)
#         losslog = f'data3/{curr_losslog}' #losslog filename str
#         #print(f'data3/{run}')
#         #print(losslog)
#         run_df = pd.read_csv(f'data3/{run}')
#         losslog_df = pd.read_csv(losslog, header=None).rename(
#             columns={0:'event', 1:'drop_unix', 2:'IP1', 3:'Port1', 4:'IP2', 5:'Port2', 6:'Proto'})
#         losslog_df['Time'] = losslog_df['drop_unix'].astype(int)
#         losslog_df = losslog_df.groupby(['Time', 'IP1', 'Port1', 'IP2', 'Port2', 'Proto']).agg(lambda x: ';'.join(x.astype(str))).reset_index()
#         df = pd.merge(run_df, losslog_df, on=['Time', 'IP1', 'Port1', 'IP2', 'Port2', 'Proto'], how="left") # merge on fivetuple key
#         # df.fillna(inplace=True, value=-1) #TODO plan implementation of dealing with null values
#         # df['event'] =  # TODO change to 3 different profiles: no drop, drop, and switch event
#         df = df[df['Proto'] == df['Proto'].mode()[0]] # selects relevant non ipv6 int(connection
        
#         ## adding labels
#         df['latency'] = int(run_labels[0])
#         df['loss'] = int(run_labels[1])
#         df['later_latency'] = int(run_labels[3])
#         df['later_loss'] = int(run_labels[4])
#         df['deterministic'] = bool(run_labels[2])
        
#         #TODO future implementation of boolean flag for when the switch happens so we know when to use later lat/loss
        
#         df.to_csv(f'{outdir}/labeled-from-{run_}_{temp_label_str}.csv') # save to temporary output directory: just merging takes a bit


NOTE TO SELF: this is wip on gen, you just copied lauras gen and are modifying it for your own use. Target for work tomorrow is getting it all to run using the main function, the model building should be trivial but gen is kicking your ass since its all the etl garbo

In [10]:
def gen(cond , tempdir, subset):
    '''Generates transformed output data aggregated in 3 files.'''
    unseen = ""
    if cond == 'seen': 
        print("transforming seen data")
    if cond == 'unseen': 
        print("transforming un seen data")
        unseen = "unseen"
    
    tempdatafiles = 'data/temp/' + tempdir # temporary data directory for training model
    tempagg = 'data/temp' # temporary data directory for training model
    
    # path = os.path.join(os.getcwd() , "outputs", "gen_temp")
    # path2 = os.path.join(os.getcwd() , "outputs")
    fnames = [ filename for filename in listdir(tempdatafiles) if filename.endswith(".csv" ) ]
    
    data, datasubset, transformed  = [], [], []
    for j in fnames:
        loc = os.path.join(os.getcwd(), 'data', "temp/" + tempdir, j)
        #print(loc)
        df_cols = genfeat(pd.read_csv(loc))
        
        #data
        time_scaled = time(df_cols)
        #print(time_scaled)
        data.append(time_scaled)
        
        #subset
        df_mid = time_scaled.iloc[60:60+subset]
        datasubset.append(df_mid)

        #transformed
        f_df = agg10(df_cols)
        transformed.append(f_df)
        
    # makes paths
    path = os.path.join(os.getcwd() , "outputs", "gen_temp")
    path2 = os.path.join(os.getcwd() , "outputs")
    
    list_to_csv(data, os.path.join(path2, unseen + "combined_all.csv"))
    print('combined_finished', sep=' ')
    list_to_csv(datasubset, os.path.join(path2, unseen + "combined_subset_6068.csv"))
    print('combined_all_finished', sep=' ')
    list_to_csv(transformed, os.path.join(path2, unseen +  "combined_transform.csv"))
    print('transformed_finished', sep=' ')
    
    return None

In [11]:
def list_to_csv(lst, filepth):
    '''takes list of pandas dataframes with similar column structure and outputs them to a single folder'''
    lst[0].to_csv(filepth, index=False)
    for i in range(1, len(lst)):
        lst[i].to_csv(filepth, index=False, header=False, mode='a')
    return

In [12]:
def genviz(cond, fname, subset):
    unseen = ""
    if cond == 'seen': 
        print("transforming seen data")
    if cond == 'unseen': 
        print("transforming un seen data")
        unseen = "unseen"

    data, datasubset, transformed  = [], [], []
    loc = os.path.join(os.getcwd(),"data3", "tempdata3", fname)
    
    t = pd.read_csv(loc)
    df_cols = genfeat(t)
    print(fname)
    run_labels = fname.split('_')[1].split('.')[0].split('-')
    print(run_labels)

    #data
    time_scaled = time(df_cols)
    data.append(time_scaled)

    #subset
    df_mid = time_scaled.iloc[:subset]
    datasubset.append(df_mid)

    #transformed
    f_df = agg10(df_cols)
    transformed.append(f_df)
    
    path = os.path.join(os.getcwd() , "outputs", "gen_temp")
    path2 = os.path.join(os.getcwd() , "outputs")
    label = run_labels[0] + '-' + run_labels[1] +  '__' + run_labels[3] + '-' + run_labels[4]
    temp_data = pd.concat(data , ignore_index=True)#.reset_index(drop = True)
    temp_data.to_csv(os.path.join(path2, unseen + "s_all_" + label + ".csv"), index = False)
    
    temp_subset = pd.concat(datasubset , ignore_index=True)
    temp_subset.to_csv(os.path.join(path2, unseen +  "s_subset_" + label + ".csv"), index = False)
    
    temp_t = pd.concat(transformed, ignore_index=True)  
    temp_t.to_csv(os.path.join(path2, unseen +  "s_transform_" + label + ".csv"), index = False)

In [13]:
def gen_helper(ct, file_lst, subset, unseen):
    """ """
    data, datasubset, transformed  = [], [], []
    for j in file_lst:
        loc = os.path.join(os.getcwd(),"data3",  "tempdata3", j)
        print(loc)
        df_cols = genfeat(pd.read_csv(loc))
        
        #data
        time_scaled = time(df_cols)
        data.append(time_scaled)
        
        #subset
        df_mid = time_scaled.iloc[60: 60+subset]
        datasubset.append(df_mid)

        #transformed
        f_df = agg10(df_cols)
        transformed.append(f_df)

    path = os.path.join(os.getcwd() , "outputs", "gen_temp")

    temp_data = pd.concat(data , ignore_index=True)#.reset_index(drop = True)
    temp_data.to_csv(os.path.join(path, unseen + "temp_all_" + str(ct) + ".csv"), index = False)
    
    temp_subset = pd.concat(datasubset , ignore_index=True)
    temp_subset.to_csv(os.path.join(path, unseen +  "temp_subset_6068_" + str(ct) + ".csv"), index = False)
    
    temp_t = pd.concat(transformed, ignore_index=True)  
    temp_t.to_csv(os.path.join(path, unseen +  "temp_transform_" + str(ct) + ".csv"), index = False)

#### eda

In [14]:
def main_eda(cond, lst, filen1, filen2, filen3):
    unseen = ''
    if cond =='unseen': 
        unseen = 'unseen'
    
    fpath1 = os.path.join(os.getcwd() , "outputs", unseen + filen1)
    df_1 = pd.read_csv(fpath1)
    fpath2 = os.path.join(os.getcwd() , "outputs", unseen + filen2)
    df_2 = pd.read_csv(fpath2)
    fpath3 = os.path.join(os.getcwd() , "outputs", unseen + filen3)
    df_3 = pd.read_csv(fpath3)
    
    plottogether(cond, lst, df_1, filen1.strip(".csv")) # trends over subset
    plottogether(cond, lst, df_3, filen3.strip(".csv")) # trends over entire data
    plotloss(cond, df_2)

    plot_correlation_matrix(cond, df_2) # correlation matrix
    plotlongest(df_3, cond, 2000, 14000)
    # below makes rest of visualizations
    plotbytes(df_3, 2000, 14000, 200, 300)
    #plot_detailed_bytes()

In [15]:
# overlaying model predictions and plotting detailed bytes

#### train

In [33]:
def test_feat(cond, df, cols, p, df_u): 
    unseen = ''
    if cond =='unseen': 
        unseen = 'unseen'
    # col is feauture comb
    # p is for loss or latency   1: loss  # 2 : latency
    X = df[cols]
    
    X2 = df_u[cols]

    if p == 1:  # flag found in test_mse
        y = df.loss
        y2 = df_u.loss
    if p == 2: 
        y = df.latency
        y2 = df_u.latency
        
    # randomly split into train and test sets, test set is 80% of data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

    if unseen == 'unseen': 
        X_test = X2
        y_test = y2
    
    clf = DecisionTreeRegressor()
    clf = clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    #acc1 = mean_squared_error(y_test, y_pred)
    acc1 = clf.score(X_test, y_test)
    
    clf2 = RandomForestRegressor(n_estimators=200, n_jobs = -1)
    clf2 = clf2.fit(X_train,y_train)
    y_pred2 = clf2.predict(X_test)
    #acc2= mean_squared_error(y_test, y_pred2)
    acc2 = clf2.score(X_test, y_test)
    
    clf3 = ExtraTreesRegressor(n_estimators=200, n_jobs = -1)
    clf3 = clf3.fit(X_train,y_train)
    y_pred3 = clf3.predict(X_test)
    #acc3= mean_squared_error(y_test, y_pred3)
    acc3 = clf3.score(X_test, y_test)
    
#     pca = PCA(n_components = 2) 
#     X_transformed = pca.fit_transform(X_train) 
#     clf4 = ExtraTreesRegressor(n_estimators=100, n_jobs = -1)
#     clf4 = clf4.fit(X_transformed, y_train)
#     newdata_transformed = pca.transform(X_test)
#     y_pred4 = clf4.predict(newdata_transformed)
#     #acc4 = mean_squared_error(y_test, y_pred4)
#     acc4 = clf4.score(X_test, y_test)
    
    clf_gbc = GradientBoostingRegressor(random_state=0, max_depth = 6, n_estimators=200)
    clf_gbc.fit(X_train, y_train)
    y_pred5 = clf_gbc.predict(X_test)
    #acc5 = mean_squared_error(y_test, y_pred5) 
    acc5 = clf_gbc.score(X_test, y_test)
    return [acc1, acc2, acc3,  acc5]


def test_mse(cond, all_comb1, all_comb2):
    unseen = ''
    if cond =='unseen': 
        unseen = 'unseen'
    filedir_unseen = os.path.join(os.getcwd(), "outputs", unseen + "combined_transform.csv")
    df_unseen = pd.read_csv(filedir_unseen)
    filedir = os.path.join(os.getcwd(), "outputs", "combined_transform.csv")
    df = pd.read_csv(filedir)
    
    all_comb1 = pd.Series(all_comb1).apply(lambda x: list(x))
    all_comb2 = pd.Series(all_comb2).apply(lambda x: list(x))
    
    dt = []
    rf = []
    et = []
    #pca = []
    gbc = []
    for i in all_comb1:
        acc_loss = test_feat(cond, df, i, 1, df_unseen)
        dt.append(acc_loss[0])  
        rf.append(acc_loss[1])  
        et.append(acc_loss[2])   
        #pca.append(acc_loss[3])   
        gbc.append(acc_loss[3])
        
    # optimze by adding a flag called losslat to avoid making two dataframes of results
    dt2 = []
    rf2 = []
    et2 = []
    #pca2 = []
    gbc2 = []
    for i in all_comb2:
        # 1 = loss
        # 2 = latency
        acc_latency = test_feat(cond, df, i, 2, df_unseen)
        dt2.append(acc_latency[0])
        rf2.append(acc_latency[1])
        et2.append(acc_latency[2]) 
        #pca2.append(acc_latency[3])
        gbc2.append(acc_latency[3])
        
    dict1 = pd.DataFrame({'feat': all_comb1, 'dt': dt, 'rf': rf, 'et': et, 'gbc': gbc})
    dict2 = pd.DataFrame({'feat2': all_comb2, 'dt2': dt2, 'rf2': rf2, 'et2': et2, 'gbc2': gbc2})
    
    path = os.path.join(os.getcwd() , "outputs")
    dict1.to_csv(os.path.join(path, unseen + "feat_df1.csv"), index = False)
    dict2.to_csv(os.path.join(path, unseen + "feat_df2.csv"), index = False)


def best_performance(cond):
    unseen = ''
    if cond == 'unseen': 
        unseen = 'unseen'
    #print("finding best loss performance")
    filedir1 = os.path.join(os.getcwd(), "outputs", unseen + "feat_df1.csv")
    df1 = pd.read_csv(filedir1)
    df1_round = df1.round(decimals = 3)
    print( "\n")
    print("Loss Performance sorted from highest to lowest metric: r2", "\n")
    print(df1_round.sort_values(by=['dt', 'rf', 'et', 'gbc'], ascending = False)[:5], "\n")
    
    #print("finding best latency performance")
    filedir2 = os.path.join(os.getcwd(), "outputs", unseen + "feat_df2.csv")
    df2 = pd.read_csv(filedir2)
    df2_round = df2.round(decimals = 3)
    print( "\n")
    print("Latency Performance sorted from highest to lowest metric: r2", "\n")
    print(df2_round.sort_values(by=['dt2', 'rf2', 'et2', 'gbc2'], ascending = False)[:5], "\n")


#### main

In [34]:
def main(targets):

    transform_config = json.load(open('config/transform.json'))
    columns = json.load(open('config/columns.json'))
    eda_config = json.load(open('config/eda.json'))
    all_config = json.load(open("config/all.json"))

    test_unseen = 'unseen'
    test_seen = 'seen'
    
    cond1 = True
    cond2 = False

    if 'data' in targets:
        """generating feat from unseen and seen data"""
        #readfilerun('data/raw/train_r', 'data/temp/tempdata_r') # TODO uncomment
        #gen(test_seen, 'tempdata_r', **transform_config)
        readfilerun('data/raw/train_c', 'data/temp/tempdata_c')
        gen(test_unseen, 'tempdata_c', **transform_config)

    if 'eda' in targets:  
        # readfiledrun and gen for seen data, refer to data target
        main_eda(test_seen, [200, 300], **eda_config)
        print("EDA saved to outputs/eda/ folder")

    if 'train' in targets:
        "trains tests in this target"
        # readfiledrun and gen for seen data, refer to data target
                
        #comb1 = getAllCombinations(1)
        #comb2 = getAllCombinations(2)
        
        #print("Testing on seen data: ")
        #test_mse(test_seen, comb1, comb2)
        best_performance(test_seen)
                        
    if "inference" in targets: 
        # readfiledrun and gen for unseen data, refer to data target
        
#         comb1 = getAllCombinations(1)
#         comb2 = getAllCombinations(2)
        
#         print("Testing on unseen data: ")
#         test_mse(test_unseen, comb1, comb2)
        best_performance(test_unseen)
            
    if "test" in targets: 
        """ runs all targets on sample data"""
        print('tba')
        # readfilerun for seen an unseen data
       
        #  main_eda(test_seen, **eda_config)
#          print("EDA saved to outputs/eda/ folder")
        
#         comb1 = getAllCombinations(1)
#         comb2 = getAllCombinations(2)
        
#         print("Testing on seen data: ")
#         test_mse(test_seen, comb1, comb2)
#         best_performance(test_seen)
        
#         print("Testing on unseen data: ")
#         test_mse(test_unseen, comb1, comb2)
#         best_performance(test_unseen)
        
    if 'all' in targets: 
        # refer to test target
        print('tba')
        
# if __name__ == '__main__':

#     targets = sys.argv[1:]
#     main(targets)

In [35]:
targets = ['data', 'eda', 'train', 'inference', 'test', 'all']
# main(targets[0])

In [36]:
main(targets[2])



Loss Performance sorted from highest to lowest metric: r2 

                                                  feat     dt     rf     et  \
59   ['mean_tdelta', 'total_pkt_sizes', 'pkt sum', ...  0.720  0.759  0.747   
91   ['mean_tdelta', 'total_pkts', 'total_pkt_sizes...  0.700  0.757  0.744   
122  ['mean_tdelta', 'total_pkt_sizes', 'number_ms'...  0.664  0.752  0.739   
93                         ['total_pkts', 'number_ms']  0.661  0.724  0.691   
104  ['total_pkts_max', 'mean_tdelta', 'total_pkt_s...  0.659  0.791  0.797   

       gbc  
59   0.769  
91   0.758  
122  0.754  
93   0.668  
104  0.774   



Latency Performance sorted from highest to lowest metric: r2 

                                       feat2    dt2    rf2    et2   gbc2
0                             ['byte_ratio']  1.000  1.000  1.000  1.000
7                ['byte_ratio', 'pkt_ratio']  1.000  1.000  0.999  1.000
1              ['byte_ratio', 'total_bytes']  0.983  0.982  0.978  0.978
3                 ['2->1Pk

In [37]:
main(targets[3])



Loss Performance sorted from highest to lowest metric: r2 

                                                 feat     dt     rf     et  \
56  ['total_pkts_max', 'mean_tdelta', 'total_pkt_s...  0.245  0.536  0.519   
49  ['mean_tdelta', 'total_pkts_max', 'pkt sum', '...  0.237  0.520  0.515   
88  ['total_pkts_max', 'mean_tdelta', 'total_pkt_s...  0.236  0.549  0.531   
19  ['mean_tdelta', 'total_pkts_max', '2->1Bytes',...  0.202  0.554  0.555   
89  ['total_pkts_max', '2->1Bytes', 'number_ms', '...  0.199  0.556  0.552   

      gbc  
56  0.429  
49  0.428  
88  0.450  
19  0.514  
89  0.506   



Latency Performance sorted from highest to lowest metric: r2 

                                                feat2    dt2    rf2    et2  \
30                                ['time_spread_min'] -2.275 -1.819 -2.132   
58  ['total_bytes', '2->1Pkts', 'pkt_ratio', 'time... -4.169 -1.837 -1.546   
55  ['pkt_ratio', 'total_bytes', 'time_spread_min'... -4.384 -1.814 -1.416   
59  ['2->1Pkts', '

git add . |
git commmit -m "message" |
git pull origin main |
git push |
user, token

In [ ]:
#listdir('data/raw/train_r')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer

label_col = 'latency'

df = pd.read_csv(os.path.join(os.getcwd() , "outputs", 'combined_transform.csv'))

indexcol = ['total_bytes','max_bytes','2->1Bytes','2->1Pkts','total_pkts', 'total_pkts_min', 'total_pkts_max', 'number_ms', 'pkt_ratio','time_spread', 'time_spread_min','time_spread_max','pkt sum','longest_seq', 'longest_seq_min', 'longest_seq_max','total_pkt_sizes','byte_ratio', 'mean_tdelta', 'max_tdelta']
len(indexcol)

X_train, X_test, y_train, y_test = train_test_split(
    df[[x for x in indexcol if x in df.columns]], df[label_col])
# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
etree = ExtraTreesRegressor(n_estimators=400, n_jobs=4)
etreeft = etree.fit(X_train,y_train)

y_pred3 = etree.predict(X_test)
acc3= mean_squared_error(y_test, y_pred3)

print(f'mse: {acc3}, r2: {etree.score(X_test, y_test)}')
feat_imp = pd.Series(index=[x for x in indexcol if x in df.columns], 
          data=etree.feature_importances_).sort_values(ascending=False)
feat_imp

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer

label_col = 'loss'

df = pd.read_csv(os.path.join(os.getcwd() , "outputs", 'combined_transform.csv'))

indexcol = ['total_bytes','max_bytes','2->1Bytes','2->1Pkts','total_pkts', 'total_pkts_min', 'total_pkts_max', 'number_ms', 'pkt_ratio','time_spread', 'time_spread_min','time_spread_max','pkt sum','longest_seq', 'longest_seq_min', 'longest_seq_max','total_pkt_sizes','byte_ratio', 'mean_tdelta', 'max_tdelta']
len(indexcol)

X_train, X_test, y_train, y_test = train_test_split(
    df[[x for x in indexcol if x in df.columns]], df[label_col])
# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
etree = ExtraTreesRegressor(n_estimators=400, n_jobs=4)
etreeft = etree.fit(X_train,y_train)

y_pred3 = etree.predict(X_test)
acc3= mean_squared_error(y_test, y_pred3)

print(f'mse: {acc3}, r2: {etree.score(X_test, y_test)}')
feat_imp = pd.Series(index=[x for x in indexcol if x in df.columns], 
          data=etree.feature_importances_).sort_values(ascending=False)
feat_imp

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.pipeline import Pipeline

path = os.path.join(os.getcwd() , "outputs")

In [ ]:
comb1 = getAllCombinations(1)
comb2 = getAllCombinations(2)
comb1

In [ ]:
#fname = 'labeled-from-data2_100-10000-true-100-10000.csv'
genviz('seen', fname, 8)

In [ ]:
newfname = 'combined_t.csv'
s_t = pd.read_csv(os.path.join(os.getcwd() , "outputs", newfname))

In [ ]:
# expanding pipeline for two predictors? mainly loss is impt, latency less so
X, y = s_t[comb1[4]], s_t['loss']
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   random_state=0)
pipe = Pipeline([('scaler', StandardScaler()), ('svc', GradientBoostingRegressor())])
 # The pipeline can be used as any other estimator
# and avoids leaking the test set into the train set


clf = GridSearchCV(GradientBoostingClassifier(), parameters,scoring=scoring,refit=False,cv=2, n_jobs=-1)


pipe.fit(X_train, y_train)
p = pipe.predict(X_test)
mean_squared_error(y_test, p)
#pipe.score(X_test, y_test)

In [ ]:
def plot_detailed_bytes(df, col='1->2Bytes', rollsec=10):
    rollcolor = '#6c2b6d'
    detailcolor = '#e98d6b'
    
    ax = plt.figure(figsize=(18,8))
    df[col].plot(title=f'{col}/s Rate', color=detailcolor)
    df[col].rolling(rollsec).mean().bfill().plot(color=rollcolor)
    plt.axvline(x=180, color='r')
    for i in df[df['event'] == 'drop'].index:
        plt.axvline(x=i, color='y', alpha=.45)
    custom_lines = [Line2D([0], [0], color=detailcolor, lw=2),
        Line2D([0], [0], color=rollcolor, lw=2),
        Line2D([0], [0], color='y', lw=2, alpha=0.45),
        Line2D([0], [0], color='r', lw=2)]
    plt.legend(custom_lines, 
               [f'{col} per Second', f'{col} per Second ({rollsec}s rolling avg)', 'Packet drop', '180s Mark'], 
               loc='upper right', framealpha=1);